In [ ]:
# Extract FineMI.zip (contains fNIRS data)
import zipfile

# Paths
finemi_zip = os.path.join(extract_dir, "FineMI.zip")
finemi_folder = os.path.join(extract_dir, "FineMI")

# Extract
if os.path.exists(finemi_zip):
    print("Extracting FineMI.zip...")
    with zipfile.ZipFile(finemi_zip, 'r') as zip_ref:
        zip_ref.extractall(finemi_folder)
    print(f"✓ Done! Extracted to: {finemi_folder}")
else:
    print(f"✗ FineMI.zip not found at: {finemi_zip}")


Extracting FineMI.zip...
✓ Done! Extracted to: /content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI


# fNIRS data

In [ ]:
# Install required packages for fNIRS processing
!pip install mne mne-nirs -q

# Define the fNIRS preprocessing function
def preprocess_subject_fnirs(subject_root):
    """Preprocess fNIRS data for one subject (subjects 2-18 only)."""
    import mne
    import mne_nirs

    fnirs_folder = os.path.join(subject_root, 'fNIRS')
    if not os.path.exists(fnirs_folder):
        raise FileNotFoundError(f"fNIRS folder not found: {fnirs_folder}")

    subject_name = os.path.basename(subject_root)
    is_subject1 = 'subject1' in subject_name.lower()
    print(f"\n{'='*60}")
    print(f"Processing {subject_name}")
    print(f"{'='*60}")

    # Find all block folders (block1, block2, ..., block8)
    block_folders = []



    if is_subject1:
        # Subject1: block1-4 (merged), then block5, block6, block7, block8
        block1_4 = os.path.join(fnirs_folder, 'block1-4')
        if os.path.exists(block1_4):
            block_folders.append(('block1-4', block1_4))

        for block_num in range(5, 9):  # Blocks 5-8
            block_folder = os.path.join(fnirs_folder, f'block{block_num}')
            if os.path.exists(block_folder):
                block_folders.append((f'block{block_num}', block_folder))
    else:
        for block_num in range(1, 9):  # Blocks 1-8
            block_folder = os.path.join(fnirs_folder, f'block{block_num}')
            if os.path.exists(block_folder):
              block_folders.append((f'block{block_num}', block_folder))

    if not block_folders:
        raise FileNotFoundError(f"No block folders found in {fnirs_folder}")

    print(f"Found {len(block_folders)} block folders")

    # Process each block
    all_teacher_epochs = []
    all_baseline_epochs = []
    all_labels = []

    for block_name, block_path in block_folders:
        print(f"\nProcessing {block_name}...")


        # Load raw fNIRS. this reads NIRS.wl1, NIRS.wl2, NIRS.hdr, NIRS.evt,...
        raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)
        print(f"  Loaded: {len(raw.ch_names)} channels")

        # Convert to optical density
        raw_od = mne.preprocessing.nirs.optical_density(raw)


        # Convert to HbO/HbR. Now we have physiologically meaningful hemoglobin signals.
        #raw_hb contains both HbO and HbR channels. hb yani hardo!
        raw_hb = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)

        # Filter 0.01-0.1 Hz. This matches exactly the filter in the paper. This removes: heartbeat, breathing, slow drift
        raw_hb.filter(l_freq=0.01, h_freq=0.1, picks='fnirs',
                      method='iir', iir_params={'order': 6, 'ftype': 'butter'},
                      verbose=False)

        # Extract events.This finds trigger codes (usually 1–8 for the 8 MI tasks
        events, event_dict = mne.events_from_annotations(raw_hb, verbose=False)

        ######################################################################
        # when we do the above line, () events, event_dict = mne.events_from_annotations(raw)) MNE returs two things:

        # 1. events , which is with the format (n_events, 3). inja, 3 yani: [event_sample_index, 0, event_id_number]
        #example: [[1250,      0,     3],    ← event 1
        #          [5300,      0,     7],    ← event 2
        #          [9100,      0,     1]]    ← event 3

        #The event happened at sample index 1250 in the raw data;, 0 is always unused. The event’s numeric label is 3, meaning MI task #3


        #2. event_dict → A dictionary describing what the annotation names mean
        #Example raw annotations might look like: "Stimulus/1", "Stimulus/3", "Stimulus/8"
        # Then event_dict will be: {'Stimulus/1': 1, 'Stimulus/3': 3, 'Stimulus/8': 8}
        #######################################################################

        # Filter to events 1-8
        valid_event_ids = {}

        #Loop through all events found
        #event name mishe key marbut be un dictionary ke esme event id ro neshun mide va adade marbut be un
        #trigger_value is MNE’s integer version of the event code
        for event_name, trigger_value in event_dict.items():
            try:
              #Try to extract a clean numeric trigger
                trigger_num = int(event_name)
            except:
              #If direct int conversion fails, parse digits using regex. Regex (\d+) extracts the first integer inside the string.
                import re
                match = re.search(r'(\d+)', event_name)
                trigger_num = int(match.group(1)) if match else 0

            if 1 <= trigger_num <= 8:
                valid_event_ids[event_name] = trigger_num

        if not valid_event_ids:
            print(f"  ⚠ No valid events found, skipping...")
            continue

        # Pick only HbO channels
        # lets only get hbo from hb signals:
        raw_hbo = raw_hb.copy().pick(picks='hbo')

        # Create teacher epochs (3.0s to 8.0s)
        #epochs_teacher will have the shape (n_trials_block, n_channels, n_timepoints)
        epochs_teacher = mne.Epochs(
            raw_hbo, events, event_id=valid_event_ids,
            tmin=3.0, tmax=8.0, baseline=None,
            preload=True, verbose=False
        )

        # Create baseline epochs (0.0s to 2.0s)
        epochs_baseline = mne.Epochs(
            raw_hbo, events, event_id=valid_event_ids,
            tmin=0.0, tmax=2.0, baseline=None,
            preload=True, verbose=False
        )
        #########
        # Extract labels and data
        #epochs teacher already contains 3-8s of data
        #we are just working with hbo!
        #because HbO usually carries a much stronger, cleaner signal for classification.
        #HbR is weaker and noisier, so it often doesn’t help much (and sometimes hurts).

        # Extract labels and data
        labels_block = epochs_teacher.events[:, -1]
        #epochs_teacher.events is an array of shape (n_trials_block, 3)
        #Recall: [sample_index, 0, event_id]
        #[:, -1] picks the last column, i.e. event_id = MI class 1–8
        teacher_data = epochs_teacher.get_data()
        baseline_data = epochs_baseline.get_data()

        print(f"  Extracted {len(teacher_data)} trials")

        all_teacher_epochs.append(teacher_data)
        all_baseline_epochs.append(baseline_data)
        all_labels.append(labels_block)

    # Concatenate all blocks

    if not all_teacher_epochs:
        raise ValueError(f"No valid epochs extracted")

    fnirs_hbo_teacher = np.concatenate(all_teacher_epochs, axis=0)
    fnirs_hbo_baseline = np.concatenate(all_baseline_epochs, axis=0)
    labels = np.concatenate(all_labels, axis=0)

    print(f"\n✓ Final shapes:")
    print(f"  Teacher: {fnirs_hbo_teacher.shape}")
    print(f"  Baseline: {fnirs_hbo_baseline.shape}")
    print(f"  Labels: {labels.shape}")

    # Save to disk
    output_folder = os.path.join(subject_root, 'preprocessed_fnirs')
    os.makedirs(output_folder, exist_ok=True)

    subject_num = subject_name.replace('subject', '')
    np.save(os.path.join(output_folder, f'subject{subject_num}_fnirs_hbo_teacher.npy'), fnirs_hbo_teacher)
    np.save(os.path.join(output_folder, f'subject{subject_num}_fnirs_hbo_baseline.npy'), fnirs_hbo_baseline)
    np.save(os.path.join(output_folder, f'subject{subject_num}_labels.npy'), labels)

    print(f"✓ Saved to {output_folder}")

    return fnirs_hbo_teacher, fnirs_hbo_baseline, labels

print("✓ fNIRS preprocessing function ready!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 84.0 MB/s eta 0:00:00
✓ fNIRS preprocessing function ready!


In [ ]:
# Install required packages for fNIRS processing
!pip install mne mne-nirs -q

# Define the fNIRS preprocessing function
def preprocess_subject_fnirs2(subject_root):
    """Preprocess fNIRS data for one subject (subjects 2-18 only)."""
    import mne
    import mne_nirs

    fnirs_folder = os.path.join(subject_root, 'fNIRS')
    if not os.path.exists(fnirs_folder):
        raise FileNotFoundError(f"fNIRS folder not found: {fnirs_folder}")

    subject_name = os.path.basename(subject_root)
    is_subject1 = subject_name.lower() == 'subject1'  # Exact match to avoid false positives (e.g., subject10)
    print(f"\n{'='*60}")
    print(f"Processing {subject_name}")
    print(f"{'='*60}")

    # Find all block folders (block1, block2, ..., block8)
    block_folders = []
    if is_subject1:
        # Subject1: block1-4 (merged), then block5, block6, block7, block8
        block1_4 = os.path.join(fnirs_folder, 'block1-4')
        if os.path.exists(block1_4):
            block_folders.append(('block1-4', block1_4))
        for block_num in range(5, 9):  # Blocks 5-8
            block_folder = os.path.join(fnirs_folder, f'block{block_num}')
            if os.path.exists(block_folder):
                block_folders.append((f'block{block_num}', block_folder))
    else:
        for block_num in range(1, 9):  # Blocks 1-8
            block_folder = os.path.join(fnirs_folder, f'block{block_num}')
            if os.path.exists(block_folder):
                block_folders.append((f'block{block_num}', block_folder))

    if not block_folders:
        raise FileNotFoundError(f"No block folders found in {fnirs_folder}")

    print(f"Found {len(block_folders)} block folders")

    # Process each block
    all_teacher_epochs = []
    all_baseline_epochs = []
    all_labels = []

    for block_name, block_path in block_folders:
        print(f"\nProcessing {block_name}...")

        # Load raw fNIRS. this reads NIRS.wl1, NIRS.wl2, NIRS.hdr, NIRS.evt,...
        raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)
        print(f"  Loaded: {len(raw.ch_names)} channels")

        # Convert to optical density
        raw_od = mne.preprocessing.nirs.optical_density(raw)

        # Convert to HbO/HbR. Now we have physiologically meaningful hemoglobin signals.
        #raw_hb contains both HbO and HbR channels. hb yani hardo!
        raw_hb = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)

        # Filter 0.01-0.1 Hz. This matches exactly the filter in the paper. This removes: heartbeat, breathing, slow drift
        raw_hb.filter(l_freq=0.01, h_freq=0.1, picks='fnirs',
                      method='iir', iir_params={'order': 6, 'ftype': 'butter'},
                      verbose=False)

        # Extract events.This finds trigger codes (usually 1–8 for the 8 MI tasks
        events, event_dict = mne.events_from_annotations(raw_hb, verbose=False)

        ######################################################################
        # when we do the above line, () events, event_dict = mne.events_from_annotations(raw)) MNE returs two things:
        # 1. events , which is with the format (n_events, 3). inja, 3 yani: [event_sample_index, 0, event_id_number]
        #example: [[1250,      0,     3],    ← event 1
        #          [5300,      0,     7],    ← event 2
        #          [9100,      0,     1]]    ← event 3
        #The event happened at sample index 1250 in the raw data;, 0 is always unused. The event’s numeric label is 3, meaning MI task #3
        #2. event_dict → A dictionary describing what the annotation names mean
        #Example raw annotations might look like: "Stimulus/1", "Stimulus/3", "Stimulus/8"
        # Then event_dict will be: {'Stimulus/1': 1, 'Stimulus/3': 3, 'Stimulus/8': 8}
        #######################################################################

        # Filter to events 1-8
        valid_event_ids = {}

        #Loop through all events found
        #event name mishe key marbut be un dictionary ke esme event id ro neshun mide va adade marbut be un
        #trigger_value is MNE’s integer version of the event code
        for event_name, trigger_value in event_dict.items():
            try:
                #Try to extract a clean numeric trigger (handle "X.0" with float)
                trigger_num = int(float(event_name))
            except:
                #If direct int conversion fails, parse digits using regex. Regex (\d+) extracts the first integer inside the string.
                import re
                match = re.search(r'(\d+)', event_name)
                trigger_num = int(match.group(1)) if match else 0

            if 1 <= trigger_num <= 8:
                valid_event_ids[event_name] = trigger_num

        if not valid_event_ids:
            print(f"  ⚠ No valid events found, skipping...")
            continue

        # Pick only HbO channels
        # lets only get hbo from hb signals:
        raw_hbo = raw_hb.copy().pick(picks='hbo')

        # Create teacher epochs (3.0s to 8.0s)
        #epochs_teacher will have the shape (n_trials_block, n_channels, n_timepoints)
        epochs_teacher = mne.Epochs(
            raw_hbo, events, event_id=valid_event_ids,
            tmin=3.0, tmax=8.0, baseline=None,
            preload=True, verbose=False
        )

        # Create baseline epochs (0.0s to 2.0s)
        epochs_baseline = mne.Epochs(
            raw_hbo, events, event_id=valid_event_ids,
            tmin=0.0, tmax=2.0, baseline=None,
            preload=True, verbose=False
        )
        #########
        # Extract labels and data
        #epochs teacher already contains 3-8s of data
        #we are just working with hbo!
        #because HbO usually carries a much stronger, cleaner signal for classification.
        #HbR is weaker and noisier, so it often doesn’t help much (and sometimes hurts).

        # Extract labels and data
        labels_block = epochs_teacher.events[:, -1]
        #epochs_teacher.events is an array of shape (n_trials_block, 3)
        #Recall: [sample_index, 0, event_id]
        #[:, -1] picks the last column, i.e. event_id = MI class 1–8
        teacher_data = epochs_teacher.get_data()
        baseline_data = epochs_baseline.get_data()

        print(f"  Extracted {len(teacher_data)} trials")

        all_teacher_epochs.append(teacher_data)
        all_baseline_epochs.append(baseline_data)
        all_labels.append(labels_block)

    # Concatenate all blocks

    if not all_teacher_epochs:
        raise ValueError(f"No valid epochs extracted")

    fnirs_hbo_teacher = np.concatenate(all_teacher_epochs, axis=0)
    fnirs_hbo_baseline = np.concatenate(all_baseline_epochs, axis=0)
    labels = np.concatenate(all_labels, axis=0)

    print(f"\n✓ Final shapes:")
    print(f"  Teacher: {fnirs_hbo_teacher.shape}")
    print(f"  Baseline: {fnirs_hbo_baseline.shape}")
    print(f"  Labels: {labels.shape}")

    # Save to disk
    output_folder = os.path.join(subject_root, 'preprocessed_fnirs')
    os.makedirs(output_folder, exist_ok=True)

    subject_num = subject_name.replace('subject', '')
    np.save(os.path.join(output_folder, f'subject{subject_num}_fnirs_hbo_teacher.npy'), fnirs_hbo_teacher)
    np.save(os.path.join(output_folder, f'subject{subject_num}_fnirs_hbo_baseline.npy'), fnirs_hbo_baseline)
    np.save(os.path.join(output_folder, f'subject{subject_num}_labels.npy'), labels)

    print(f"✓ Saved to {output_folder}")

    return fnirs_hbo_teacher, fnirs_hbo_baseline, labels

print("✓ fNIRS preprocessing function ready!")

✓ fNIRS preprocessing function ready!


# lets check if this is gonna work on annotations

In [ ]:
import mne
import re
import os

def inspect_fnirs_annotations(block_path):
    print(f"\n=== Inspecting annotations in {block_path} ===")

    # Load raw NIRx data
    raw = mne.io.read_raw_nirx(block_path, preload=False, verbose=False)

    # Get annotations
    ann = raw.annotations

    if len(ann) == 0:
        print("⚠ No annotations found!")
        return

    print(f"Found {len(ann)} annotations.\n")

    # Loop through annotations and show raw text plus extracted numbers
    for i, (desc, onset) in enumerate(zip(ann.description, ann.onset)):
        # Try to extract digits
        match = re.search(r'(\d+)', desc)
        extracted = match.group(1) if match else None

        print(f"{i+1}. Raw annotation: {desc}")
        print(f"   Onset time (sec): {onset}")
        print(f"   Extracted trigger number: {extracted}")
        print()

# Example usage:
inspect_fnirs_annotations("/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject11/fNIRS/block5")



=== Inspecting annotations in /content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject11/fNIRS/block5 ===
Found 40 annotations.

1. Raw annotation: 3.0
   Onset time (sec): 222.72
   Extracted trigger number: 3

2. Raw annotation: 4.0
   Onset time (sec): 240.768
   Extracted trigger number: 4

3. Raw annotation: 8.0
   Onset time (sec): 259.712
   Extracted trigger number: 8

4. Raw annotation: 7.0
   Onset time (sec): 278.784
   Extracted trigger number: 7

5. Raw annotation: 2.0
   Onset time (sec): 297.728
   Extracted trigger number: 2

6. Raw annotation: 7.0
   Onset time (sec): 317.824
   Extracted trigger number: 7

7. Raw annotation: 3.0
   Onset time (sec): 337.792
   Extracted trigger number: 3

8. Raw annotation: 5.0
   Onset time (sec): 357.76
   Extracted trigger number: 5

9. Raw annotation: 4.0
   Onset time (sec): 376.832
   Extracted trigger number: 4

10. Raw annotation: 5.0
   Onset time (sec): 395.776
   Extracted trigger number: 5

11. Raw an

/tmp/ipython-input-2944705597.py:9: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"2023Äê2ÔÂ16ÈÕ""11:27:53.586"'.
  raw = mne.io.read_raw_nirx(block_path, preload=False, verbose=False)


# so yes annotation names are clean!

In [ ]:
# Process a Subject fNIRS data just to see how it looks like

# Set up path to FineMI folder
finemi_root = os.path.join(extract_dir, "FineMI", "FineMI")
subject10_fnirs_path = os.path.join(finemi_root, "subject10")

if os.path.exists(subject10_fnirs_path):
    print("Processing Subject 1 fNIRS data...")
    fnirs_teacher, fnirs_baseline, fnirs_labels = preprocess_subject_fnirs2(subject10_fnirs_path)

    print(f"\n✓ Subject 10 fNIRS preprocessing complete!")
    print(f"  Ready for alignment with EEG data")
else:
    print(f"Subject 10 folder not found at: {subject10_fnirs_path}")
    print("Make sure FineMI.zip has been extracted.")


Processing Subject 1 fNIRS data...

Processing subject10
Found 8 block folders

Processing block1...


/tmp/ipython-input-2975180500.py:51: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"2023Äê2ÔÂ13ÈÕ""15:24:57.733"'.
  raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)


  Loaded: 48 channels
  Extracted 41 trials

Processing block2...


/tmp/ipython-input-2975180500.py:51: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"2023Äê2ÔÂ13ÈÕ""15:43:15.684"'.
  raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)


  Loaded: 48 channels
  Extracted 41 trials

Processing block3...


...

  Loaded: 48 channels
  Extracted 41 trials

Processing block7...


/tmp/ipython-input-2975180500.py:51: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"2023Äê2ÔÂ13ÈÕ""17:17:57.226"'.
  raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)


  Loaded: 48 channels
  Extracted 40 trials

Processing block8...


/tmp/ipython-input-2975180500.py:51: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"2023Äê2ÔÂ13ÈÕ""17:36:20.074"'.
  raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)


  Loaded: 48 channels
  Extracted 40 trials

✓ Final shapes:
  Teacher: (324, 24, 40)
  Baseline: (324, 24, 17)
  Labels: (324,)
✓ Saved to /content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject10/preprocessed_fnirs

✓ Subject 10 fNIRS preprocessing complete!
  Ready for alignment with EEG data


In [ ]:
# Process fNIRS data for ALL subjects 1-18

# Set up path to FineMI folder
finemi_root = os.path.join(extract_dir, "FineMI", "FineMI")

# Find all subject folders (only directories, not files)
import glob
import re
import numpy as np

all_subjects = [f for f in glob.glob(os.path.join(finemi_root, "subject*"))
                if os.path.isdir(f)]

# Sort by subject number (not alphabetically)
def get_subject_number(path):
    """Extract subject number from path for proper sorting."""
    basename = os.path.basename(path)
    match = re.search(r'subject(\d+)', basename, re.IGNORECASE)
    return int(match.group(1)) if match else 0

# Sort by subject number (include all subjects 1-18)
all_subjects_sorted = sorted(all_subjects, key=get_subject_number)
subjects_to_process = all_subjects_sorted

print(f"Found {len(subjects_to_process)} subjects to process (subjects 1-18)")
print(f"Subjects: {[os.path.basename(s) for s in subjects_to_process]}")
print("="*60)

# Process all subjects
successful = 0
failed = 0

for subject_path in subjects_to_process:
    subject_name = os.path.basename(subject_path)
    print(f"\n{'='*60}")
    print(f"Processing {subject_name} ({subjects_to_process.index(subject_path) + 1}/{len(subjects_to_process)})")
    print(f"{'='*60}")

    try:
        fnirs_teacher, fnirs_baseline, fnirs_labels = preprocess_subject_fnirs2(subject_path)
        successful += 1
        print(f"\n✓ {subject_name} processed successfully!")
    except Exception as e:
        failed += 1
        print(f"\n✗ Error processing {subject_name}: {str(e)}")
        continue

print(f"\n{'='*60}")
print("Processing Summary")
print(f"{'='*60}")
print(f"✓ Successful: {successful}/{len(subjects_to_process)}")
print(f"✗ Failed: {failed}/{len(subjects_to_process)}")
print(f"\nAll preprocessed fNIRS data saved to Drive!")


Found 18 subjects to process (subjects 1-18)
Subjects: ['subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8', 'subject9', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14', 'subject15', 'subject16', 'subject17', 'subject18']

Processing subject1 (1/18)

Processing subject1
Found 5 block folders

Processing block1-4...


/tmp/ipython-input-2975180500.py:51: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"2021Äê8ÔÂ18ÈÕ""15:23:07.439"'.
  raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)


  Loaded: 48 channels
  Extracted 200 trials

Processing block5...


/tmp/ipython-input-2975180500.py:51: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"2021Äê8ÔÂ18ÈÕ""16:43:47.025"'.
  raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)


  Loaded: 48 channels
  Extracted 40 trials

Processing block6...


...

  Loaded: 48 channels
  Extracted 40 trials

Processing block7...


/tmp/ipython-input-2975180500.py:51: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"2023Äê3ÔÂ17ÈÕ""16:03:10.176"'.
  raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)


  Loaded: 48 channels
  Extracted 40 trials

Processing block8...


/tmp/ipython-input-2975180500.py:51: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"2023Äê3ÔÂ17ÈÕ""16:16:32.227"'.
  raw = mne.io.read_raw_nirx(block_path, preload=True, verbose=False)


  Loaded: 48 channels
  Extracted 40 trials

✓ Final shapes:
  Teacher: (320, 24, 40)
  Baseline: (320, 24, 17)
  Labels: (320,)
✓ Saved to /content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject18/preprocessed_fnirs

✓ subject18 processed successfully!

Processing Summary
✓ Successful: 18/18
✗ Failed: 0/18

All preprocessed fNIRS data saved to Drive!


In [ ]:
"""
Simple EEG Data Loader for Multi-Joint Upper Limb MI Dataset

This module loads EEG data from .cnt files and prepares it for analysis.
"""

import os
import glob
import re
#file handling + pattern matching + regex.
!pip install mne
import numpy as np
import mne


# Map trigger numbers (1-8) to MI task names
MI_TASKS = {
    1: 'hand_open_close',
    2: 'wrist_flexion_extension',
    3: 'wrist_abduction_adduction',
    4: 'elbow_pronation_supination',
    5: 'elbow_flexion_extension',
    6: 'shoulder_pronation_supination',
    7: 'shoulder_abduction_adduction',
    8: 'shoulder_flexion_extension'
}

#load_subject_eeg(...): from CNT files → clean continuous EEG
#Given a path like /.../subject10, return one Raw object that:
#has all 8 blocks concatenated,non-EEG channels marked as misc, filtered 4–40 Hz,resampled to 250 Hz
def load_subject_eeg(subject_path, target_sampling_rate=250.0, verbose=True):

    # Step 1: Find EEG folder and detect subject1
    #subject_path should contain an EEG folder. If not → error
    eeg_folder = os.path.join(subject_path, 'EEG')
    if not os.path.exists(eeg_folder):
        raise FileNotFoundError(f"EEG folder not found: {eeg_folder}")

    # Step 2: Check if this is Subject1 (has special file naming)
    subject_name = os.path.basename(subject_path.rstrip('/'))
    is_subject1 = 'subject1' in subject_name.lower()

    # Step 3: Find all .cnt files
    cnt_files = glob.glob(os.path.join(eeg_folder, '*.cnt'))
    cnt_files.extend(glob.glob(os.path.join(eeg_folder, '*.CNT')))

    if not cnt_files:
        raise FileNotFoundError(f"No .cnt files found in {eeg_folder}")

    if verbose:
        print(f"Found {len(cnt_files)} EEG blocks for {subject_name}")

    # Step 4: Sort files in correct order
    if is_subject1:
        # Subject1: block1-4.cnt comes first, then block5, block6, block7, block8
        def get_block_number(filename):
            name = os.path.basename(filename).lower()
            if 'block1-4' in name:
                return 1  # First
            elif 'block5' in name:
                return 5
            elif 'block6' in name:
                return 6
            elif 'block7' in name:
                return 7
            elif 'block8' in name:
                return 8
            else:
                match = re.search(r'block(\d+)', name)
                return int(match.group(1)) if match else 999

        cnt_files = sorted(cnt_files, key=get_block_number)
    else:
        # Other subjects: block1, block2, block3, ..., block8
        def get_block_number(filename):
            name = os.path.basename(filename).lower()
            match = re.search(r'block(\d+)', name)
            return int(match.group(1)) if match else 999

        cnt_files = sorted(cnt_files, key=get_block_number)

    if verbose:
        print("Loading blocks in order:")
        for i, f in enumerate(cnt_files, 1):
            print(f"  {i}. {os.path.basename(f)}")

    # Step 5: Load each block, set channel types for non-EEG channels
    #Load each block, mark misc channels
    all_blocks = []
    for cnt_file in cnt_files:
        if verbose:
            print(f"  Loading {os.path.basename(cnt_file)}...")
        block_data = mne.io.read_raw_cnt(cnt_file, preload=True, verbose=False)

        # Mark known non-EEG channels as 'misc' (adapt list if needed)
        misc_channels = ['CB1', 'CB2', 'HEO', 'VEO', 'EKG', 'EMG']
        present_misc = [ch for ch in misc_channels if ch in block_data.ch_names]
        if present_misc:
            block_data.set_channel_types({ch: 'misc' for ch in present_misc})

        # OPTIONAL: set montage if appropriate for your 64-channel layout
        # block_data.set_montage('standard_1020', on_missing='ignore')

        all_blocks.append(block_data)

    # Ensure sampling frequency consistent, then concatenate. we don't mix if sfreq different
    sfreqs = {blk.info['sfreq'] for blk in all_blocks}
    if len(sfreqs) > 1:
        raise RuntimeError(f"Blocks have different sampling rates: {sfreqs}")

    # Step 6: Combine all blocks into one
    #If just one block, hamoono estefade kon dige
    #age bishtar darim, hameye block haro concatenate kon
    if len(all_blocks) == 1:
        combined_data = all_blocks[0]
    else:
        combined_data = mne.concatenate_raws(all_blocks, preload=True, verbose=False)

    # Step 7: Apply bandpass filter (4–40 Hz) - removes noise
    if verbose:
        print("Applying bandpass filter (4–40 Hz)...")
    combined_data.filter(l_freq=4.0, h_freq=40.0, picks='eeg', verbose=False)

    # Step 8: Resample to target rate (250 Hz) - reduces data size
    current_rate = combined_data.info['sfreq']
    if current_rate != target_sampling_rate:
        if verbose:
            print(f"Resampling from {current_rate} Hz to {target_sampling_rate} Hz...")
        combined_data.resample(target_sampling_rate, verbose=False)

    # Step 9: Print summary
    if verbose:
        n_channels = len(combined_data.ch_names)
        duration = combined_data.times[-1]
        print("\n✓ Loaded successfully!")
        print(f"  Channels: {n_channels} (EEG + misc)")
        print(f"  Sampling rate: {combined_data.info['sfreq']} Hz")
        print(f"  Duration: {duration:.1f} seconds")

    return combined_data


def extract_epochs(raw, start_time=-0.5, end_time=4.0, baseline=None, verbose=True):
    """
    Extract individual trials (epochs) from continuous EEG data.

    Each trial is aligned to when the cue appears (trigger 1–8).

    Parameters
    ----------
    raw : mne.io.Raw
        Continuous EEG data
    start_time : float
        Start time relative to cue (seconds, default: -0.5 = 500 ms before)
    end_time : float
        End time relative to cue (seconds, default: 4.0 = 4 seconds after)
    baseline : tuple or None
        Baseline correction interval (in seconds, relative to event).
        If None, uses (start_time, 0.0) when start_time < 0, else (None, 0.0).
    verbose : bool
        Print progress messages (default: True)

    Returns
    -------
    epochs : mne.Epochs
        Extracted trials
    labels : np.ndarray
        Task labels (0–7) for each trial
    """
    # Step 1: Find all events (triggers) in the data
    events, event_dict = mne.events_from_annotations(raw, verbose=False)
    print(raw.annotations.description[:20])
    print(raw.annotations.onset[:20])


    if verbose:
        print(f"Found {len(events)} events in the data")
        print(f"Event types: {list(event_dict.keys())}")

    # Step 2: Build event_id mapping for triggers 1–8
    # Keep event codes as they are (usually 1..8, matching events[:, 2])
    valid_events = {}
    for event_name, code in event_dict.items():
        # Prefer simple numeric event names like '1', '2', ..., '8'
        if event_name.isdigit():
            trigger_num = int(event_name)
        else:
            match = re.search(r'(\d+)', event_name)
            if not match:
                continue
            trigger_num = int(match.group(1))

        if 1 <= trigger_num <= 8:
            valid_events[event_name] = code  # <= keep original code

    if not valid_events:
        raise ValueError("No valid events (triggers 1–8) found!")

    if verbose:
        print(f"Using {len(valid_events)} event types (event_id): {valid_events}")

    # Step 3: Determine baseline window
    if baseline is None:
        if start_time < 0:
            baseline = (start_time, 0.0)  # pre-cue baseline
        else:
            baseline = (None, 0.0)

    # Step 4: Extract epochs
    epochs = mne.Epochs(
        raw,
        events,
        event_id=valid_events,
        tmin=start_time,
        tmax=end_time,
        baseline=baseline,
        preload=True,
        verbose=False
    )

    # Step 5: Get labels
    # epochs.events[:, -1] contains the event codes (typically 1..8).
    # Convert event codes 1..8 to ML labels 0..7.
    event_codes = epochs.events[:, -1]
    labels = event_codes - 1

    if verbose:
        n_trials = len(epochs)
        data_shape = epochs.get_data().shape
        print("\n✓ Extracted epochs")
        print(f"  Trials: {n_trials}")
        print(f"  Shape: {data_shape} = (trials, channels, timepoints)")
        print(f"  Event codes in epochs: {np.unique(event_codes)}")
        print(f"  Labels (0–7): {labels.min()} to {labels.max()}")
        print(f"  Label counts: {np.bincount(labels)}")

    return epochs, labels


def labels_to_task_names(labels):
    """
    Convert integer labels (0–7) to MI task name strings using MI_TASKS.
    """
    labels = np.asarray(labels, dtype=int)
    task_names = np.array([MI_TASKS[l + 1] for l in labels])  # 0–7 -> 1–8
    return task_names


def epochs_to_numpy(epochs):
    """
    Convert MNE Epochs to a simple NumPy array.

    Returns
    -------
    data : np.ndarray
        Array of shape (trials, channels, timepoints)
    """
    return epochs.get_data()


def save_epochs_numpy(epochs, labels, out_path):
    """
    Save epochs and labels as a compressed NumPy file.

    Parameters
    ----------
    epochs : mne.Epochs
        Epoched EEG data
    labels : array-like
        Integer labels (0–7)
    out_path : str
        Output file path (e.g., 'subject1_eeg_epochs.npz')
    """
    data = epochs.get_data()
    labels = np.asarray(labels, dtype=int)
    np.savez_compressed(out_path, data=data, labels=labels)
    print(f"Saved: {out_path}")
    print(f"  data shape: {data.shape}")
    print(f"  labels shape: {labels.shape}")


# Example usage
if __name__ == "__main__":
    print("=" * 60)
    print("Example: Loading Subject 1")
    print("=" * 60)

    # Update this path to your dataset location
    dataset_root = "/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI"
    subject1_path = os.path.join(dataset_root, "subject1")

    if os.path.exists(subject1_path):
        # Load EEG
        raw = load_subject_eeg(subject1_path, verbose=True)

        # Extract trials
        epochs, labels = extract_epochs(raw, verbose=True)

        # Convert to NumPy
        data = epochs_to_numpy(epochs)
        print(f"\nNumPy array shape: {data.shape}")

        # Optional: see task names for the first 10 trials
        task_names = labels_to_task_names(labels)
        print("First 10 task names:", task_names[:10])

        # Optional: save to disk
        out_file = os.path.join(dataset_root, "subject1_eeg_epochs_4_40hz_250hz.npz")
        save_epochs_numpy(epochs, labels, out_file)

    else:
        print(f"Subject folder not found: {subject1_path}")
        print("Update the path above to your dataset location.")


Example: Loading Subject 1
Found 5 EEG blocks for subject1
Loading blocks in order:
  1. block1-4.cnt
  2. block5.cnt
  3. block6.cnt
  4. block7.cnt
  5. block8.cnt
  Loading block1-4.cnt...


/tmp/ipython-input-760139961.py:93: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  block_data = mne.io.read_raw_cnt(cnt_file, preload=True, verbose=False)
/tmp/ipython-input-760139961.py:99: RuntimeWarning: The unit for channel(s) CB1, CB2, EKG, EMG, HEO, VEO has changed from V to NA.
  block_data.set_channel_types({ch: 'misc' for ch in present_misc})


  Loading block5.cnt...


/tmp/ipython-input-760139961.py:93: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  block_data = mne.io.read_raw_cnt(cnt_file, preload=True, verbose=False)
/tmp/ipython-input-760139961.py:99: RuntimeWarning: The unit for channel(s) CB1, CB2, EKG, EMG, HEO, VEO has changed from V to NA.
  block_data.set_channel_types({ch: 'misc' for ch in present_misc})


  Loading block6.cnt...


...

  Loading block7.cnt...


/tmp/ipython-input-760139961.py:93: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  block_data = mne.io.read_raw_cnt(cnt_file, preload=True, verbose=False)
/tmp/ipython-input-760139961.py:99: RuntimeWarning: The unit for channel(s) CB1, CB2, EKG, EMG, HEO, VEO has changed from V to NA.
  block_data.set_channel_types({ch: 'misc' for ch in present_misc})


  Loading block8.cnt...


/tmp/ipython-input-760139961.py:93: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  block_data = mne.io.read_raw_cnt(cnt_file, preload=True, verbose=False)
/tmp/ipython-input-760139961.py:99: RuntimeWarning: The unit for channel(s) CB1, CB2, EKG, EMG, HEO, VEO has changed from V to NA.
  block_data.set_channel_types({ch: 'misc' for ch in present_misc})


Applying bandpass filter (4–40 Hz)...
Resampling from 1000.0 Hz to 250.0 Hz...

✓ Loaded successfully!
  Channels: 68 (EEG + misc)
  Sampling rate: 250.0 Hz
  Duration: 7813.5 seconds
['1' '8' '6' '2' '3' '1' '3' '4' '8' '7' '3' '1' '4' '8' '3' '7' '5' '6'
 '8' '2']
[ 78.845  98.859 117.876 137.89  156.906 176.921 194.939 213.956 232.972
 250.99  271.004 290.021 310.035 330.05  350.065 368.083 387.099 405.117
 423.134 443.15 ]
Found 360 events in the data
Event types: [np.str_('1'), np.str_('2'), np.str_('3'), np.str_('4'), np.str_('5'), np.str_('6'), np.str_('7'), np.str_('8')]
Using 8 event types (event_id): {np.str_('1'): 1, np.str_('2'): 2, np.str_('3'): 3, np.str_('4'): 4, np.str_('5'): 5, np.str_('6'): 6, np.str_('7'): 7, np.str_('8'): 8}

✓ Extracted epochs
  Trials: 360
  Shape: (360, 68, 1126) = (trials, channels, timepoints)
  Event codes in epochs: [1 2 3 4 5 6 7 8]
  Labels (0–7): 0 to 7
  Label counts: [45 45 45 45 45 45 45 45]

NumPy array shape: (360, 68, 1126)
First 10 

In [ ]:
# === Process ALL subjects (2–18) ===
import os
import glob

# If your functions are defined in the same notebook, you don't need this import.
# If they are in eeg_loader.py, uncomment the next line:
# from eeg_loader import load_subject_eeg, extract_epochs, save_epochs_numpy

# Root where "FineMI/FineMI/subjectX" folders are located
dataset_root = "/content/drive/MyDrive/multi_joint_mi_dataset/extracted"
finemi_root = os.path.join(dataset_root, "FineMI", "FineMI")

# Find all subject folders: subject1, subject2, ..., subject18
all_subjects = sorted(glob.glob(os.path.join(finemi_root, "subject*")))

print(f"Found {len(all_subjects)} subject folders:")
for s in all_subjects:
    print(" ", os.path.basename(s))

print("\n===========================================")

# Loop over all subjects, skipping subject1 (already done)
for subject_path in all_subjects:
    subject_name = os.path.basename(subject_path)

    # Skip subject1 if you already processed it
    if subject_name.lower() == "subject1":
        print(f"Skipping {subject_name} (already processed).")
        continue

    print("\n===========================================")
    print(f"Processing {subject_name} ...")
    print("===========================================")

    try:
        # 1) Load continuous EEG
        raw = load_subject_eeg(subject_path, verbose=True)

        # 2) Extract epochs and labels
        epochs, labels = extract_epochs(raw, verbose=True)

        # 3) Save as compressed NumPy file
        out_file = os.path.join(
            finemi_root,
            f"{subject_name}_eeg_epochs_4_40hz_250hz.npz"
        )
        save_epochs_numpy(epochs, labels, out_file)

    except Exception as e:
        print(f"\n✗ Error while processing {subject_name}: {e}")
        # Continue with the next subject
        continue

print("\n✅ Done processing all subjects (except subject1 if skipped).")


Found 19 subject folders:
  subject1
  subject10
  subject11
  subject12
  subject13
  subject14
  subject15
  subject16
  subject17
  subject18
  subject1_eeg_epochs_4_40hz_250hz.npz
  subject2
  subject3
  subject4
  subject5
  subject6
  subject7
  subject8
  subject9

Skipping subject1 (already processed).

Processing subject10 ...
Found 8 EEG blocks for subject10
Loading blocks in order:
  1. block1.cnt
  2. block2.cnt
  3. block3.cnt
  4. block4.cnt
  5. block5.cnt
  6. block6.cnt
  7. block7.cnt
  8. block8.cnt
  Loading block1.cnt...


/tmp/ipython-input-1484056838.py:113: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  block_data = mne.io.read_raw_cnt(cnt_file, preload=True, verbose=False)
/tmp/ipython-input-1484056838.py:119: RuntimeWarning: The unit for channel(s) CB1, CB2, EKG, EMG, HEO, VEO has changed from V to NA.
  block_data.set_channel_types({ch: 'misc' for ch in present_misc})


  Loading block2.cnt...


/tmp/ipython-input-1484056838.py:113: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  block_data = mne.io.read_raw_cnt(cnt_file, preload=True, verbose=False)
/tmp/ipython-input-1484056838.py:119: RuntimeWarning: The unit for channel(s) CB1, CB2, EKG, EMG, HEO, VEO has changed from V to NA.
  block_data.set_channel_types({ch: 'misc' for ch in present_misc})


  Loading block3.cnt...


...

  Loading block7.cnt...


/tmp/ipython-input-1484056838.py:113: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  block_data = mne.io.read_raw_cnt(cnt_file, preload=True, verbose=False)
/tmp/ipython-input-1484056838.py:119: RuntimeWarning: The unit for channel(s) CB1, CB2, EKG, EMG, HEO, VEO has changed from V to NA.
  block_data.set_channel_types({ch: 'misc' for ch in present_misc})


  Loading block8.cnt...


/tmp/ipython-input-1484056838.py:113: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  block_data = mne.io.read_raw_cnt(cnt_file, preload=True, verbose=False)
/tmp/ipython-input-1484056838.py:119: RuntimeWarning: The unit for channel(s) CB1, CB2, EKG, EMG, HEO, VEO has changed from V to NA.
  block_data.set_channel_types({ch: 'misc' for ch in present_misc})


Applying bandpass filter (4–40 Hz)...
Resampling from 1000.0 Hz to 250.0 Hz...

✓ Loaded successfully!
  Channels: 68 (EEG + misc)
  Sampling rate: 250.0 Hz
  Duration: 6474.2 seconds
Found 320 events in the data
Event types: [np.str_('1'), np.str_('2'), np.str_('3'), np.str_('4'), np.str_('5'), np.str_('6'), np.str_('7'), np.str_('8')]
Using 8 event types (event_id): {np.str_('1'): 1, np.str_('2'): 2, np.str_('3'): 3, np.str_('4'): 4, np.str_('5'): 5, np.str_('6'): 6, np.str_('7'): 7, np.str_('8'): 8}

✓ Extracted epochs
  Trials: 320
  Shape: (320, 68, 1126) = (trials, channels, timepoints)
  Event codes in epochs: [1 2 3 4 5 6 7 8]
  Labels (0–7): 0 to 7
  Label counts: [40 40 40 40 40 40 40 40]
Saved: /content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject9_eeg_epochs_4_40hz_250hz.npz
  data shape: (320, 68, 1126)
  labels shape: (320,)

✅ Done processing all subjects (except subject1 if skipped).


# **Trial Alignment**

In [ ]:
def align_eeg_fnirs_trials(eeg_labels, fnirs_labels, eeg_data=None, fnirs_data=None, verbose=True):
    """
    Align EEG and fNIRS trials by scanning integer shifts between label sequences
    and then keeping only trials where labels match.

    This handles:
      - Different trial counts across modalities
      - Constant offsets (e.g., extra initial fNIRS trials)
      - Per-subject differences like your subject11 (+8), subject14 (0), subject10 (+2)

    Returned data/labels are already aligned and filtered.
    """
    eeg_labels = np.asarray(eeg_labels)
    fnirs_labels = np.asarray(fnirs_labels)

    n_eeg = len(eeg_labels)
    n_fnirs = len(fnirs_labels)

    if verbose:
        print("="*60)
        print("Trial Alignment: EEG ↔ fNIRS (with shift scan)")
        print("="*60)
        print(f"EEG trials:  {n_eeg}")
        print(f"fNIRS trials:{n_fnirs}")

    # ---------- 1) Find best shift ----------
    def match_rate_for_shift(eeg, fnirs, shift):
        if shift >= 0:
            fn_sub = fnirs[shift:]
            eeg_sub = eeg[:len(fn_sub)]
        else:
            eeg_sub = eeg[-shift:]
            fn_sub = fnirs[:len(eeg_sub)]
        n = min(len(eeg_sub), len(fn_sub))
        if n == 0:
            return 0.0, 0, None
        eeg_sub = eeg_sub[:n]
        fn_sub = fn_sub[:n]
        matches = (eeg_sub == fn_sub)
        rate = float(matches.mean() * 100.0)
        return rate, n, matches

    max_shift = 12
    best_shift = None
    best_rate = -1.0
    best_matches = None
    best_n = 0

    for s in range(-max_shift, max_shift + 1):
        rate, n, matches = match_rate_for_shift(eeg_labels, fnirs_labels, s)
        if rate > best_rate:
            best_rate = rate
            best_shift = s
            best_matches = matches
            best_n = n

    if verbose:
        print(f"\nBest shift: {best_shift:+d}")
        print(f"Raw match rate over overlapping trials: {best_rate:.1f}%")
        print(f"Overlapping trials at best shift: {best_n}")

        if best_rate < 70:
            print("⚠ WARNING: low match rate; this subject may have misaligned or corrupted trials.")

    # ---------- 2) Build overlapping index ranges ----------
    if best_shift >= 0:
        fnirs_start = best_shift
        eeg_start = 0
    else:
        fnirs_start = 0
        eeg_start = -best_shift

    n_overlap = min(n_eeg - eeg_start, n_fnirs - fnirs_start)
    if n_overlap <= 0:
        raise RuntimeError("No overlapping trials after applying best shift; check data.")

    eeg_idx_full = np.arange(eeg_start, eeg_start + n_overlap)
    fnirs_idx_full = np.arange(fnirs_start, fnirs_start + n_overlap)

    # best_matches is defined over the overlapping part in match_rate_for_shift,
    # so we just truncate to n_overlap to be safe.
    label_matches_full = best_matches[:n_overlap]
    mismatch_indices = np.where(~label_matches_full)[0]

    # ---------- 3) Keep only matching-label trials ----------
    keep_mask = label_matches_full
    eeg_idx = eeg_idx_full[keep_mask]
    fnirs_idx = fnirs_idx_full[keep_mask]

    aligned_eeg_labels = eeg_labels[eeg_idx]
    aligned_fnirs_labels = fnirs_labels[fnirs_idx]

    assert np.array_equal(aligned_eeg_labels, aligned_fnirs_labels)

    if verbose:
        print(f"\nAfter filtering mismatches:")
        print(f"  Aligned trials kept: {len(eeg_idx)}")
        print(f"  Label mismatches removed: {len(mismatch_indices)}")
        print(f"  Final labels are identical across modalities: {np.array_equal(aligned_eeg_labels, aligned_fnirs_labels)}")

        # Show some mismatch details (based on overlapping window, not final kept data)
        if len(mismatch_indices) > 0:
            print("\n  Mismatch details in overlapping region (first 10):")
            for idx in mismatch_indices[:10]:
                print(f"    Overlap trial {idx}: EEG={eeg_labels[eeg_idx_full[idx]]}, "
                      f"fNIRS={fnirs_labels[fnirs_idx_full[idx]]}")
            if len(mismatch_indices) > 10:
                print(f"    ... and {len(mismatch_indices) - 10} more")

    # ---------- 4) Align data if provided ----------
    aligned_eeg_data = None
    aligned_fnirs_data = None

    if eeg_data is not None:
        aligned_eeg_data = eeg_data[eeg_idx]
        if verbose:
            print(f"\n  EEG data aligned:   {aligned_eeg_data.shape}")

    if fnirs_data is not None:
        aligned_fnirs_data = fnirs_data[fnirs_idx]
        if verbose:
            print(f"  fNIRS data aligned: {aligned_fnirs_data.shape}")

    # ---------- 5) Alignment info ----------
    alignment_info = {
        'n_eeg_trials': int(n_eeg),
        'n_fnirs_trials': int(n_fnirs),
        'best_shift': int(best_shift),
        'n_overlapping_trials': int(best_n),
        'raw_match_rate': float(best_rate),    # before dropping mismatches
        'n_aligned_trials': int(len(eeg_idx)), # after dropping mismatches
        'n_label_mismatches_removed': int(len(mismatch_indices)),
        'match_rate': float(best_rate),        # keep this name for your downstream code
        'mismatch_indices': mismatch_indices.tolist(),
    }

    if verbose:
        print(f"\n{'='*60}")

    return aligned_eeg_labels, aligned_fnirs_labels, aligned_eeg_data, aligned_fnirs_data, alignment_info


In [ ]:

from compute_best_alignment_shift import load_and_align_subject
subject_folder = "/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject10"
aligned = load_and_align_subject(subject_folder, verbose=True)
print("Final match rate:", aligned["alignment_info"]["match_rate"])
print("Labels identical:", np.array_equal(aligned["eeg_labels"], aligned["fnirs_labels"]))


DEBUG: Received subject_path = '/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject10'

Loading and aligning: subject10
✓ Loaded EEG: (320, 68, 1126), labels: (320,)
  From: subject10_eeg_epochs_4_40hz_250hz.npz
✓ Loaded fNIRS: (324, 24, 40), labels: (324,)
  From: preprocessed_fnirs/
  EEG label range:   0 to 7
  fNIRS label range: 0 to 7
Trial Alignment: EEG ↔ fNIRS (with shift scan)
EEG trials:   320
fNIRS trials: 324

Best shift: +2
Raw match rate over overlapping trials: 42.8%
Overlapping trials at best shift: 320
⚠ WARNING: low match rate; this subject may have misaligned or corrupted trials.

After filtering mismatches:
  Aligned trials kept: 137
  Label mismatches removed: 183
  Final labels identical across modalities: True

  Mismatch details in overlapping region (first 10):
    Overlap trial 0: EEG=3, fNIRS=0
    Overlap trial 1: EEG=0, fNIRS=3
    Overlap trial 2: EEG=3, fNIRS=1
    Overlap trial 3: EEG=1, fNIRS=4
    Overlap trial 4: EEG=4, fNIRS

In [ ]:
eeg_test = os.path.join(finemi_root, "subject2_eeg_epochs_4_40hz_250hz.npz")
fnirs_teacher_test = os.path.join(subject2_path, "preprocessed_fnirs", "subject2_fnirs_hbo_teacher.npy")
fnirs_labels_test = os.path.join(subject2_path, "preprocessed_fnirs", "subject2_labels.npy")

print("EEG file exists:", os.path.exists(eeg_test), eeg_test)
print("fNIRS teacher exists:", os.path.exists(fnirs_teacher_test), fnirs_teacher_test)
print("fNIRS labels exists:", os.path.exists(fnirs_labels_test), fnirs_labels_test)


NameError: name 'subject2_path' is not defined

In [ ]:
# ---- 1) Set paths ----
finemi_root = "/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI"
subject10_path = os.path.join(finemi_root, "subject11")

print("FineMI root:", finemi_root)
print("Subject 10 path:", subject10_path)

# ---- 2) Run alignment for subject10 ----
aligned_data = load_and_align_subject(
    subject_path=subject10_path,
    finemi_root=finemi_root,
    verbose=True,   # keep True now to see shift details
)

# ---- 3) Print a small summary ----
print("\n" + "="*60)
print(f"Alignment Test Summary for {aligned_data['subject_name']}")
print("="*60)
print("Aligned trials:", aligned_data["alignment_info"]["n_aligned_trials"])
print("Raw label match rate:", aligned_data['alignment_info']['raw_match_rate'])
print("EEG data shape:", aligned_data["eeg_data"].shape)
print("fNIRS data shape:", aligned_data["fnirs_data"].shape)
print("EEG labels shape:", aligned_data["eeg_labels"].shape)
print("fNIRS labels shape:", aligned_data["fnirs_labels"].shape)

labels_match = np.array_equal(aligned_data["eeg_labels"], aligned_data["fnirs_labels"])
print("Final labels identical:", labels_match)


FineMI root: /content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI
Subject 10 path: /content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject11
DEBUG: Received subject_path = '/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject11'

Loading and aligning: subject11
✓ Loaded EEG: (320, 68, 1126), labels: (320,)
  From: subject11_eeg_epochs_4_40hz_250hz.npz
✓ Loaded fNIRS: (328, 24, 40), labels: (328,)
  From: preprocessed_fnirs/
  EEG label range:   0 to 7
  fNIRS label range: 0 to 7
Trial Alignment: EEG ↔ fNIRS (with shift scan)
EEG trials:   320
fNIRS trials: 328

Best shift: +8
Raw match rate over overlapping trials: 87.5%
Overlapping trials at best shift: 320

After filtering mismatches:
  Aligned trials kept: 280
  Label mismatches removed: 40
  Final labels identical across modalities: True

  Mismatch details in overlapping region (first 10):
    Overlap trial 0: EEG=4, fNIRS=5
    Overlap trial 1: EEG=5, fNIRS=2
    Ove

**This means fnirs and eeg alignment function is working!**

# Step 9: Align All Subjects and Save Clean Datasets

Align EEG and fNIRS data for all subjects and save clean, aligned datasets to disk.


In [ ]:
import numpy as np
import os
import glob
import re

# Set up paths
finemi_root = os.path.join(extract_dir, "FineMI", "FineMI")

# Find all subject folders (only directories)
all_subjects = [f for f in glob.glob(os.path.join(finemi_root, "subject*"))
                if os.path.isdir(f)]

# Sort by subject number (not alphabetically)
def get_subject_number(path):
    """Extract subject number from path for proper sorting."""
    basename = os.path.basename(path)
    match = re.search(r'subject(\d+)', basename, re.IGNORECASE)
    return int(match.group(1)) if match else 0

all_subjects_sorted = sorted(all_subjects, key=get_subject_number)

print(f"Found {len(all_subjects_sorted)} subjects to align")
print(f"Subjects: {[os.path.basename(s) for s in all_subjects_sorted]}")
print("="*60)

# Create output directory for aligned datasets
output_dir = os.path.join(extract_dir, "aligned_datasets")
os.makedirs(output_dir, exist_ok=True)
print(f"\nAligned datasets will be saved to: {output_dir}")

# You can optionally define a warning threshold
RAW_MATCH_WARNING = 70.0   # % raw match rate before dropping mismatches

# Process all subjects
successful = 0
failed = 0
failed_subjects = []

for idx, subject_path in enumerate(all_subjects_sorted, start=1):
    subject_name = os.path.basename(subject_path)
    subject_num = subject_name.replace('subject', '')

    print(f"\n{'='*60}")
    print(f"Processing {subject_name} ({idx}/{len(all_subjects_sorted)})")
    print(f"{'='*60}")

    try:
        # Load and align (shift-aware)
        aligned_data = load_and_align_subject(
            subject_path=subject_path,
            finemi_root=finemi_root,
            verbose=True  # set to False if output is too long
        )

        alignment_info = aligned_data['alignment_info']
        n_aligned = alignment_info.get('n_aligned_trials', None)
        raw_match_rate = alignment_info.get('raw_match_rate', None)
        best_shift = alignment_info.get('best_shift', None)

        print(f"\nSummary for {subject_name}:")
        print(f"  Best shift:           {best_shift}")
        print(f"  Raw match rate:       {raw_match_rate:.1f}% (before dropping mismatches)")
        print(f"  Aligned trials kept:  {n_aligned}")

        if raw_match_rate is not None and raw_match_rate < RAW_MATCH_WARNING:
            print(f"  ⚠ WARNING: Raw match rate < {RAW_MATCH_WARNING:.1f}% "
                  f"→ this subject may have problematic alignment (e.g., subject10).")

        # Prepare clean dataset
        clean_labels = aligned_data['eeg_labels']  # after alignment and mismatch removal

        # Verify final labels match exactly (they should)
        labels_equal = np.array_equal(aligned_data['eeg_labels'], aligned_data['fnirs_labels'])
        if not labels_equal:
            print(f"  ⚠ Warning: Final EEG and fNIRS labels are not identical for {subject_name}")
            print(f"    EEG labels range:   {aligned_data['eeg_labels'].min()}–{aligned_data['eeg_labels'].max()}")
            print(f"    fNIRS labels range: {aligned_data['fnirs_labels'].min()}–{aligned_data['fnirs_labels'].max()}")
        else:
            print(f"  ✓ Final labels are identical for {subject_name}")

        # Save clean dataset
        output_file = os.path.join(output_dir, f"{subject_name}_aligned.npz")
        np.savez_compressed(
            output_file,
            eeg_data=aligned_data['eeg_data'],
            fnirs_data=aligned_data['fnirs_data'],
            labels=clean_labels,
            alignment_info=alignment_info
        )

        print(f"\n✓ Saved aligned dataset: {os.path.basename(output_file)}")
        print(f"  EEG shape:   {aligned_data['eeg_data'].shape}")
        print(f"  fNIRS shape: {aligned_data['fnirs_data'].shape}")
        print(f"  Labels shape:{clean_labels.shape}")
        print(f"  Label range: {clean_labels.min()} to {clean_labels.max()}")

        successful += 1

    except Exception as e:
        failed += 1
        failed_subjects.append(subject_name)
        print(f"\n✗ Error processing {subject_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*60}")
print("Processing Summary")
print(f"{'='*60}")
print(f"✓ Successful: {successful}/{len(all_subjects_sorted)}")
print(f"✗ Failed:     {failed}/{len(all_subjects_sorted)}")

if failed_subjects:
    print(f"\nFailed subjects: {failed_subjects}")

print(f"\nAll aligned datasets saved to: {output_dir}")
print("Each file contains:")
print("  - eeg_data:   (trials, eeg_channels, timepoints)")
print("  - fnirs_data: (trials, fnirs_channels, timepoints)")
print("  - labels:     (trials,) - 0–7 for 8 MI tasks")
print("  - alignment_info: dict with alignment statistics (best_shift, raw_match_rate, etc.)")


Found 18 subjects to align
Subjects: ['subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8', 'subject9', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14', 'subject15', 'subject16', 'subject17', 'subject18']

Aligned datasets will be saved to: /content/drive/MyDrive/multi_joint_mi_dataset/extracted/aligned_datasets

Processing subject1 (1/18)
DEBUG: Received subject_path = '/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject1'

Loading and aligning: subject1
✓ Loaded EEG: (360, 68, 1126), labels: (360,)
  From: subject1_eeg_epochs_4_40hz_250hz.npz
✓ Loaded fNIRS: (360, 24, 40), labels: (360,)
  From: preprocessed_fnirs/
  EEG label range:   0 to 7
  fNIRS label range: 0 to 7
Trial Alignment: EEG ↔ fNIRS (with shift scan)
EEG trials:   360
fNIRS trials: 360

Best shift: +0
Raw match rate over overlapping trials: 100.0%
Overlapping trials at best shift: 360

After filtering mismatches:
  Aligned trials kept:

In [ ]:
import numpy as np
import os

# === CHANGE THIS for different subjects ===
subject_num = 14  # try 10, 11, 14 etc.
base_dir = "/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI"
subject_name = f"subject{subject_num}"
subject_path = os.path.join(base_dir, subject_name)

# EEG file in FineMI/FineMI root
eeg_file = os.path.join(base_dir, f"{subject_name}_eeg_epochs_4_40hz_250hz.npz")

# fNIRS files in subject/preprocessed_fnirs
fnirs_folder = os.path.join(subject_path, "preprocessed_fnirs")
fnirs_labels_file = os.path.join(fnirs_folder, f"{subject_name}_labels.npy")

print("EEG file:", eeg_file, "exists:", os.path.exists(eeg_file))
print("fNIRS labels file:", fnirs_labels_file, "exists:", os.path.exists(fnirs_labels_file))

eeg_data = np.load(eeg_file)
eeg_labels = eeg_data["labels"]

fnirs_labels_raw = np.load(fnirs_labels_file)

# Remap fNIRS 1–8 -> 0–7 if needed
if fnirs_labels_raw.min() == 1 and fnirs_labels_raw.max() == 8:
    fnirs_labels = fnirs_labels_raw - 1
else:
    fnirs_labels = fnirs_labels_raw.copy()

print("\nEEG labels shape:", eeg_labels.shape, "unique:", np.unique(eeg_labels))
print("fNIRS labels shape:", fnirs_labels.shape, "unique:", np.unique(fnirs_labels))

def match_rate_for_shift(eeg, fnirs, shift):
    """
    Positive shift => shift fNIRS forward (drop first 'shift' trials)
    Negative shift => shift EEG forward (drop first '-shift' trials)
    """
    if shift >= 0:
        fn_sub = fnirs[shift:]
        eeg_sub = eeg[:len(fn_sub)]
    else:
        eeg_sub = eeg[-shift:]
        fn_sub = fnirs[:len(eeg_sub)]
    n = min(len(eeg_sub), len(fn_sub))
    if n == 0:
        return 0.0, 0
    eeg_sub = eeg_sub[:n]
    fn_sub = fn_sub[:n]
    return float((eeg_sub == fn_sub).mean() * 100.0), n

# Scan a range of shifts
max_shift = 12  # you can increase if needed
best_shift = None
best_rate = -1
best_n = 0

print("\nScanning shifts...")
for s in range(-max_shift, max_shift + 1):
    rate, n = match_rate_for_shift(eeg_labels, fnirs_labels, s)
    print(f"Shift {s:+d}: match_rate={rate:.1f}% over {n} overlapping trials")
    if rate > best_rate:
        best_rate = rate
        best_shift = s
        best_n = n

print("\nBest shift:", best_shift)
print("Best match rate:", best_rate)
print("Overlapping trials used:", best_n)

# Show first 20 labels for the best shift for visual sanity-check
rate, n = match_rate_for_shift(eeg_labels, fnirs_labels, best_shift)
if best_shift >= 0:
    fn_sub = fnirs_labels[best_shift:best_shift + 20]
    eeg_sub = eeg_labels[:20]
else:
    eeg_sub = eeg_labels[-best_shift:-best_shift + 20]
    fn_sub = fnirs_labels[:20]

print("\nFirst 20 EEG labels:   ", eeg_sub)
print("First 20 fNIRS (shifted):", fn_sub)


EEG file: /content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject14_eeg_epochs_4_40hz_250hz.npz exists: True
fNIRS labels file: /content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject14/preprocessed_fnirs/subject14_labels.npy exists: True

EEG labels shape: (320,) unique: [0 1 2 3 4 5 6 7]
fNIRS labels shape: (322,) unique: [0 1 2 3 4 5 6 7]

Scanning shifts...
Shift -12: match_rate=11.4% over 308 overlapping trials
Shift -11: match_rate=9.7% over 309 overlapping trials
Shift -10: match_rate=11.3% over 310 overlapping trials
Shift -9: match_rate=10.6% over 311 overlapping trials
Shift -8: match_rate=10.6% over 312 overlapping trials
Shift -7: match_rate=11.2% over 313 overlapping trials
Shift -6: match_rate=8.9% over 314 overlapping trials
Shift -5: match_rate=12.7% over 315 overlapping trials
Shift -4: match_rate=9.2% over 316 overlapping trials
Shift -3: match_rate=10.7% over 317 overlapping trials
Shift -2: match_rate=11.0% over 318 overla

In [ ]:

import numpy as np

sub10 = "/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject11"
aligned_10 = load_and_align_subject(sub10, verbose=True)

print("\nFinal match rate:", aligned_10["alignment_info"]["match_rate"])
print("Labels identical:", np.array_equal(aligned_10["eeg_labels"], aligned_10["fnirs_labels"]))


DEBUG: Received subject_path = '/content/drive/MyDrive/multi_joint_mi_dataset/extracted/FineMI/FineMI/subject11'

Loading and aligning: subject11
✓ Loaded EEG: (320, 68, 1126), labels: (320,)
  From: subject11_eeg_epochs_4_40hz_250hz.npz
✓ Loaded fNIRS: (328, 24, 40), labels: (328,)
  From: preprocessed_fnirs/
  EEG label range:   0 to 7
  fNIRS label range: 0 to 7
Trial Alignment: EEG ↔ fNIRS (with shift scan)
EEG trials:   320
fNIRS trials: 328

Best shift: +8
Raw match rate over overlapping trials: 87.5%
Overlapping trials at best shift: 320

After filtering mismatches:
  Aligned trials kept: 280
  Label mismatches removed: 40
  Final labels identical across modalities: True

  Mismatch details in overlapping region (first 10):
    Overlap trial 0: EEG=4, fNIRS=5
    Overlap trial 1: EEG=5, fNIRS=2
    Overlap trial 2: EEG=2, fNIRS=7
    Overlap trial 3: EEG=7, fNIRS=2
    Overlap trial 4: EEG=2, fNIRS=7
    Overlap trial 5: EEG=7, fNIRS=4
    Overlap trial 6: EEG=4, fNIRS=1
    Ove